In [2]:
import SimpleITK as sitk
from radiomics import featureextractor
import json
import glob
import pandas as pd
import re
import os
import logging
from tqdm.notebook import tqdm
from pathlib import Path
import pypickle

In [3]:
!python3 preprocessing.py --username="daniil.tikhonov" --data="/home/daniil.tikhonov/mri/data/Lumiere" --saveto="/home/daniil.tikhonov/mri/new_atlas_registered"

--- Starting Preprocessing: Registering all scans to MNI atlas ---
Registering cases to atlas: 100%|█████████| 361/361 [2:32:49<00:00, 25.40s/case]
--- Preprocessing complete. All files saved to atlas_mapping directory. ---


In [13]:
DATA_PATH = Path(os.getcwd()) / "data" / "Lumiere"
PATIENTS_JSON = DATA_PATH / "patients.json"
NEW_IMAGES_PATH = Path(os.getcwd()) / "new_atlas_registered"
NEW_SCANS_PATH = NEW_IMAGES_PATH / "scans"
NEW_MASKS_PATH = NEW_IMAGES_PATH / "segmentations"

with open(PATIENTS_JSON) as f:
    patients_dict = json.load(f)

In [14]:
os.listdir(NEW_SCANS_PATH)

['patient_018_case_01_baseline_T1.nii.gz',
 'patient_077_case_05_baseline_T1.nii.gz',
 'patient_033_case_02_followup_FLAIR.nii.gz',
 'patient_007_case_03_baseline_T1.nii.gz',
 'patient_054_case_06_baseline_T2.nii.gz',
 'patient_031_case_06_followup_FLAIR.nii.gz',
 'patient_032_case_05_baseline_T2.nii.gz',
 'patient_078_case_02_followup_T2.nii.gz',
 'patient_071_case_05_followup_T2.nii.gz',
 'patient_059_case_02_baseline_T1CE.nii.gz',
 'patient_054_case_05_baseline_FLAIR.nii.gz',
 'patient_042_case_03_baseline_T2.nii.gz',
 'patient_029_case_01_baseline_T2.nii.gz',
 'patient_030_case_03_followup_T1CE.nii.gz',
 'patient_024_case_04_baseline_T1CE.nii.gz',
 'patient_075_case_03_followup_T2.nii.gz',
 'patient_015_case_01_baseline_T1CE.nii.gz',
 'patient_029_case_05_followup_FLAIR.nii.gz',
 'patient_031_case_03_baseline_FLAIR.nii.gz',
 'patient_062_case_03_baseline_T1CE.nii.gz',
 'patient_059_case_07_baseline_T2.nii.gz',
 'patient_063_case_06_followup_T2.nii.gz',
 'patient_029_case_10_followu

In [15]:
patients_dict

{'patient_001': {'case_01': {'baseline': './images/Patient-001_week-000-2',
   'baseline_seg': './segmentations/Patient-001_week-000-2.nii.gz',
   'baseline_registered': './images_registered/Patient-001_week-000-2_reg',
   'baseline_seg_registered': './segmentations_registered/Patient-001_week-000-2_reg.nii.gz',
   'followup': './images/Patient-001_week-044',
   'followup_seg': './segmentations/Patient-001_week-044.nii.gz',
   'followup_registered': './images_registered/Patient-001_week-044_reg',
   'followup_seg_registered': './segmentations_registered/Patient-001_week-044_reg.nii.gz',
   'response': 2},
  'case_02': {'baseline': './images/Patient-001_week-000-2',
   'baseline_seg': './segmentations/Patient-001_week-000-2.nii.gz',
   'baseline_registered': './images_registered/Patient-001_week-000-2_reg',
   'baseline_seg_registered': './segmentations_registered/Patient-001_week-000-2_reg.nii.gz',
   'followup': './images/Patient-001_week-056',
   'followup_seg': './segmentations/Pati

In [18]:
for patient_id, visits in patients_dict.items():
    print(patient_id)
    print(visits)
    break

patient_001
{'case_01': {'baseline': './images/Patient-001_week-000-2', 'baseline_seg': './segmentations/Patient-001_week-000-2.nii.gz', 'baseline_registered': './images_registered/Patient-001_week-000-2_reg', 'baseline_seg_registered': './segmentations_registered/Patient-001_week-000-2_reg.nii.gz', 'followup': './images/Patient-001_week-044', 'followup_seg': './segmentations/Patient-001_week-044.nii.gz', 'followup_registered': './images_registered/Patient-001_week-044_reg', 'followup_seg_registered': './segmentations_registered/Patient-001_week-044_reg.nii.gz', 'response': 2}, 'case_02': {'baseline': './images/Patient-001_week-000-2', 'baseline_seg': './segmentations/Patient-001_week-000-2.nii.gz', 'baseline_registered': './images_registered/Patient-001_week-000-2_reg', 'baseline_seg_registered': './segmentations_registered/Patient-001_week-000-2_reg.nii.gz', 'followup': './images/Patient-001_week-056', 'followup_seg': './segmentations/Patient-001_week-056.nii.gz', 'followup_registere

In [48]:
new_patients_dict = {}

for patient_id, patient_data in patients_dict.items():
    visits = []
    for case_id, case_data in patient_data.items():
        visits += [{case_id: {
            "baseline_T1": f"./{patient_id}_{case_id}_baseline_T1.nii.gz",
            "baseline_T1CE": f"./{patient_id}_{case_id}_baseline_T1CE.nii.gz",
            "baseline_T2": f"./{patient_id}_{case_id}_baseline_T2.nii.gz",
            "baseline_FLAIR": f"./{patient_id}_{case_id}_baseline_FLAIR.nii.gz",
            "baseline_seg": f"./{patient_id}_{case_id}_baseline_seg.nii.gz",
            "followup_T1": f"./{patient_id}_{case_id}_followup_T1.nii.gz",
            "followup_T1CE": f"./{patient_id}_{case_id}_followup_T1CE.nii.gz",
            "followup_T2": f"./{patient_id}_{case_id}_followup_T2.nii.gz",
            "followup_FLAIR": f"./{patient_id}_{case_id}_followup_FLAIR.nii.gz",
            "followup_seg": f"./{patient_id}_{case_id}_followup_seg.nii.gz",
            "label": case_data["response"],
        }}]
    
    if visits:
        for visit in visits:
            visit_data = visit[list(visit.keys())[0]]
            for key in visit_data:
                if isinstance(visit_data[key], Path):
                    visit_data[key] = str(visit_data[key])
        new_patients_dict[patient_id] = {
            case_id: visit_data for visit in visits for case_id, visit_data in visit.items()
        }

In [49]:
new_patients_dict["patient_001"]

{'case_01': {'baseline_T1': './patient_001_case_01_baseline_T1.nii.gz',
  'baseline_T1CE': './patient_001_case_01_baseline_T1CE.nii.gz',
  'baseline_T2': './patient_001_case_01_baseline_T2.nii.gz',
  'baseline_FLAIR': './patient_001_case_01_baseline_FLAIR.nii.gz',
  'baseline_seg': './patient_001_case_01_baseline_seg.nii.gz',
  'followup_T1': './patient_001_case_01_followup_T1.nii.gz',
  'followup_T1CE': './patient_001_case_01_followup_T1CE.nii.gz',
  'followup_T2': './patient_001_case_01_followup_T2.nii.gz',
  'followup_FLAIR': './patient_001_case_01_followup_FLAIR.nii.gz',
  'followup_seg': './patient_001_case_01_followup_seg.nii.gz',
  'label': 2},
 'case_02': {'baseline_T1': './patient_001_case_02_baseline_T1.nii.gz',
  'baseline_T1CE': './patient_001_case_02_baseline_T1CE.nii.gz',
  'baseline_T2': './patient_001_case_02_baseline_T2.nii.gz',
  'baseline_FLAIR': './patient_001_case_02_baseline_FLAIR.nii.gz',
  'baseline_seg': './patient_001_case_02_baseline_seg.nii.gz',
  'followup_

In [50]:
PATIENTS_JSON_NEW = DATA_PATH / "patients_registered.json"
PATIENTS_JSON_NEW

PosixPath('/home/daniil.tikhonov/mri/data/Lumiere/patients_registered.json')

In [51]:
with open(PATIENTS_JSON_NEW, "w") as f:
    json.dump(new_patients_dict, f, indent=4)

In [47]:
with open(PATIENTS_JSON, "w") as f:
    json.dump(patients_dict, f, indent=4)

In [25]:
patients_dict

{'patient_001': {'case_01': {'baseline': './images/Patient-001_week-000-2',
   'baseline_seg': './segmentations/Patient-001_week-000-2.nii.gz',
   'baseline_registered': './images_registered/Patient-001_week-000-2_reg',
   'baseline_seg_registered': './segmentations_registered/Patient-001_week-000-2_reg.nii.gz',
   'followup': './images/Patient-001_week-044',
   'followup_seg': './segmentations/Patient-001_week-044.nii.gz',
   'followup_registered': './images_registered/Patient-001_week-044_reg',
   'followup_seg_registered': './segmentations_registered/Patient-001_week-044_reg.nii.gz',
   'response': 2},
  'case_02': {'baseline': './images/Patient-001_week-000-2',
   'baseline_seg': './segmentations/Patient-001_week-000-2.nii.gz',
   'baseline_registered': './images_registered/Patient-001_week-000-2_reg',
   'baseline_seg_registered': './segmentations_registered/Patient-001_week-000-2_reg.nii.gz',
   'followup': './images/Patient-001_week-056',
   'followup_seg': './segmentations/Pati

In [8]:
patients_dict['patient_001']

{'case_01': {'baseline': './images/Patient-001_week-000-2',
  'baseline_seg': './segmentations/Patient-001_week-000-2.nii.gz',
  'baseline_registered': './images_registered/Patient-001_week-000-2_reg',
  'baseline_seg_registered': './segmentations_registered/Patient-001_week-000-2_reg.nii.gz',
  'followup': './images/Patient-001_week-044',
  'followup_seg': './segmentations/Patient-001_week-044.nii.gz',
  'followup_registered': './images_registered/Patient-001_week-044_reg',
  'followup_seg_registered': './segmentations_registered/Patient-001_week-044_reg.nii.gz',
  'response': 2},
 'case_02': {'baseline': './images/Patient-001_week-000-2',
  'baseline_seg': './segmentations/Patient-001_week-000-2.nii.gz',
  'baseline_registered': './images_registered/Patient-001_week-000-2_reg',
  'baseline_seg_registered': './segmentations_registered/Patient-001_week-000-2_reg.nii.gz',
  'followup': './images/Patient-001_week-056',
  'followup_seg': './segmentations/Patient-001_week-056.nii.gz',
  'f

In [2]:
USERNAME = "daniil.tikhonov"
MNI_TEMPLATE_PATH = Path("/home") / USERNAME / "fsl" / "data" / "standard" / "MNI152_T1_1mm_brain.nii.gz"

if not os.path.exists(MNI_TEMPLATE_PATH):
    raise FileNotFoundError(f"MNI Template not found at: {MNI_TEMPLATE_PATH}. Please update the path.")

data_dir = Path("/home") / USERNAME / "mri" / "data" / "Lumiere"
patients_file = os.path.join(data_dir, "patients.json")

atlas_dir = Path(os.getcwd()) / "atlas_mapping"
atlas_scans_dir = atlas_dir / "scans"
atlas_segs_dir = atlas_dir / "segmentations"
os.makedirs(atlas_scans_dir, exist_ok=True)
os.makedirs(atlas_segs_dir, exist_ok=True)

In [ ]:
def get_patient_id_from_path(path):
    match = re.search(r'Patient-(\d+)', path)
    if match: return f'patient_{match.group(1).zfill(3)}'
    else: raise ValueError(f"No valid patient ID found in path: {path}")

def load_patients(patients_file):
    with open(patients_file) as f: return json.load(f)

In [ ]:
def register_to_atlas(fixed_image, moving_t1_image, moving_other_image, is_segmentation=False):
    """
    Registers an image to the atlas space using a transform derived from its corresponding T1 scan.
    
    Args:
        fixed_image (sitk.Image): The MNI atlas template image.
        moving_t1_image (sitk.Image): The patient's T1 scan (used to calculate the transform).
        moving_other_image (sitk.Image): The image to be transformed (can be T1c, T2, FLAIR, or seg).
        is_segmentation (bool): If True, use Nearest Neighbor interpolation.
        
    Returns:
        sitk.Image: The resampled/registered image.
    """
    # Use a robust registration method
    registration_method = sitk.ImageRegistrationMethod()
    
    # Similarity metric
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.01)
    
    # Interpolator
    registration_method.SetInterpolator(sitk.sitkLinear)
    
    # Optimizer
    registration_method.SetOptimizerAsGradientDescent(
        learningRate=0.33,
        numberOfIterations=300,
        convergenceMinimumValue=1e-6,
        convergenceWindowSize=10
    )
    registration_method.SetOptimizerScalesFromPhysicalShift()
    
    # Setup for the transform
    initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                          moving_t1_image, 
                                                          sitk.Euler3DTransform(), 
                                                          sitk.CenteredTransformInitializerFilter.GEOMETRY)
    
    registration_method.SetInitialTransform(initial_transform, inPlace=False)
    
    # Execute the registration to find the transform
    final_transform = registration_method.Execute(sitk.Cast(fixed_image, sitk.sitkFloat32), 
                                                  sitk.Cast(moving_t1_image, sitk.sitkFloat32))

    # --- Apply the found transform to the 'moving_other_image' ---
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(fixed_image) # Resample to the same grid as the atlas
    resampler.SetTransform(final_transform)
    
    # Use appropriate interpolation
    if is_segmentation:
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resampler.SetInterpolator(sitk.sitkLinear)
        
    resampler.SetDefaultPixelValue(0) # Pad with black
    
    return resampler.Execute(moving_other_image)


# === Preprocessing Step to Create Atlas-Registered Files ===

def preprocess_and_register_all_scans(patients, data_dir):
    """
    Iterates through all scans, registers them to the MNI atlas, and saves them.
    This version correctly handles inconsistencies in patients.json.
    """
    print("--- Starting Preprocessing: Registering all scans to MNI atlas ---")
    fixed_image = sitk.ReadImage(MNI_TEMPLATE_PATH, sitk.sitkFloat32)
    total_cases = sum(len(cases) for cases in patients.values())
    pbar = tqdm(total=total_cases, desc="Registering cases to atlas", unit="case")

    # The outer `pid` from `patients.items()` is unreliable.
    for unreliable_pid, cases in patients.items():
        for cid, meta in cases.items():
            try:
                true_pid = get_patient_id_from_path(meta['baseline_registered'])

                # --- Process BASELINE scans ---
                baseline_t1_atlas_path = os.path.join(atlas_scans_dir, f"{true_pid}_{cid}_baseline_T1.nii.gz")
                if not os.path.exists(baseline_t1_atlas_path):
                    baseline_t1_path = f"{data_dir}/{meta['baseline_registered'].replace('./', '')}/{meta['baseline_registered'].replace('./images_registered/', '')}_0000.nii.gz"
                    moving_t1_baseline = sitk.ReadImage(baseline_t1_path, sitk.sitkFloat32)
                    
                    for mri_type_idx, mri_type_name in enumerate(['T1', 'T1CE', 'T2', 'FLAIR']):
                        image_path = f"{data_dir}/{meta['baseline_registered'].replace('./', '')}/{meta['baseline_registered'].replace('./images_registered/', '')}_{mri_type_idx:04d}.nii.gz"
                        moving_image = sitk.ReadImage(image_path, sitk.sitkFloat32)
                        registered_image = register_to_atlas(fixed_image, moving_t1_baseline, moving_image)
                        # Use 'true_pid' for saving the file.
                        output_path = os.path.join(atlas_scans_dir, f"{true_pid}_{cid}_baseline_{mri_type_name}.nii.gz")
                        sitk.WriteImage(registered_image, output_path)

                    seg_path = f"{data_dir}/{meta['baseline_seg_registered'].replace('./', '')}"
                    moving_seg = sitk.ReadImage(seg_path)
                    registered_seg = register_to_atlas(fixed_image, moving_t1_baseline, moving_seg, is_segmentation=True)
                    # Use 'true_pid' for saving the file.
                    output_seg_path = os.path.join(atlas_segs_dir, f"{true_pid}_{cid}_baseline_seg.nii.gz")
                    sitk.WriteImage(registered_seg, output_seg_path)

                # --- Process FOLLOWUP scans ---
                followup_t1_atlas_path = os.path.join(atlas_scans_dir, f"{true_pid}_{cid}_followup_T1.nii.gz")
                if not os.path.exists(followup_t1_atlas_path):
                    followup_t1_path = f"{data_dir}/{meta['followup_registered'].replace('./', '')}/{meta['followup_registered'].replace('./images_registered/', '')}_0000.nii.gz"
                    moving_t1_followup = sitk.ReadImage(followup_t1_path, sitk.sitkFloat32)

                    for mri_type_idx, mri_type_name in enumerate(['T1', 'T1CE', 'T2', 'FLAIR']):
                        image_path = f"{data_dir}/{meta['followup_registered'].replace('./', '')}/{meta['followup_registered'].replace('./images_registered/', '')}_{mri_type_idx:04d}.nii.gz"
                        moving_image = sitk.ReadImage(image_path, sitk.sitkFloat32)
                        registered_image = register_to_atlas(fixed_image, moving_t1_followup, moving_image)
                        output_path = os.path.join(atlas_scans_dir, f"{true_pid}_{cid}_followup_{mri_type_name}.nii.gz")
                        sitk.WriteImage(registered_image, output_path)

                    seg_path = f"{data_dir}/{meta['followup_seg_registered'].replace('./', '')}"
                    moving_seg = sitk.ReadImage(seg_path)
                    registered_seg = register_to_atlas(fixed_image, moving_t1_followup, moving_seg, is_segmentation=True)
                    output_seg_path = os.path.join(atlas_segs_dir, f"{true_pid}_{cid}_followup_seg.nii.gz")
                    sitk.WriteImage(registered_seg, output_seg_path)

            except Exception as e:
                print(f"ERROR processing case {unreliable_pid}/{cid}. True Patient: {get_patient_id_from_path(meta.get('baseline_registered', '')) if meta.get('baseline_registered') else 'Unknown'}. Error: {e}")
            finally:
                pbar.update(1)

    pbar.close()
    print("--- Preprocessing complete. All files saved to atlas_mapping directory. ---")

In [ ]:
# Load patient data
patients = load_patients(patients_file)

# STEP 1: Run the one-time preprocessing to register all images to the atlddas
# You can comment this out after it has been run once.
preprocess_and_register_all_scans(patients, data_dir)